In [1]:
# %% [markdown]
# شبیه‌سازی Hamming(7,4) + QPSK + OFDM / OFDMA

# %%
import numpy as np

# ===============================
# 1. Hamming(7,4) encoder / decoder
# ===============================

def hamming74_encode(bits):
    """
    Hamming(7,4) systematic encoder
    input:  1D array/list of bits {0,1}
    output: coded bits (len multiple of 7)
    """
    bits = np.array(bits).astype(int)
    if len(bits) % 4 != 0:
        pad = 4 - (len(bits) % 4)
        bits = np.concatenate([bits, np.zeros(pad, dtype=int)])
    data = bits.reshape(-1, 4)

    # Generator matrix G (4x7) for Hamming(7,4) systematic form [I4 | P]
    G = np.array([
        [1, 0, 0, 0, 0, 1, 1],
        [0, 1, 0, 0, 1, 0, 1],
        [0, 0, 1, 0, 1, 1, 0],
        [0, 0, 0, 1, 1, 1, 1]
    ], dtype=int)

    codewords = (data @ G) % 2
    return codewords.reshape(-1)


def hamming74_decode(received):
    """
    Hamming(7,4) hard-decision decoder
    input:  coded bits (len multiple of 7)
    output: estimated data bits (length multiple of 4)
    """
    r = np.array(received).astype(int)
    assert len(r) % 7 == 0, "Length of received must be multiple of 7."
    r = r.reshape(-1, 7)

    # Parity-check matrix H (3x7)
    H = np.array([
        [1, 0, 1, 0, 1, 0, 1],
        [0, 1, 1, 0, 0, 1, 1],
        [0, 0, 0, 1, 1, 1, 1]
    ], dtype=int)

    syndromes = (H @ r.T) % 2  # 3 x n
    syndromes = syndromes.T    # n x 3

    corrected = r.copy()

    # syndrome → error position (1..7), 0 means no error
    syndrome_table = {
        (0, 0, 0): 0,
        (1, 0, 0): 1,
        (0, 1, 0): 2,
        (1, 1, 0): 3,
        (0, 0, 1): 4,
        (1, 0, 1): 5,
        (0, 1, 1): 6,
        (1, 1, 1): 7,
    }

    for i, s in enumerate(syndromes):
        idx = syndrome_table.get(tuple(s.tolist()), 0)
        if idx != 0:
            corrected[i, idx - 1] ^= 1  # flip that bit

    data = corrected[:, 0:4].reshape(-1)
    return data


# ===============================
# 2. QPSK Mod / Demod (Gray)
# ===============================

def qpsk_modulate(bits):
    """
    Gray-coded QPSK
    bits pair: (b0,b1)
    00 ->  1 + 1j
    01 ->  1 - 1j
    11 -> -1 - 1j
    10 -> -1 + 1j
    (normalized by sqrt(2))
    """
    bits = np.array(bits).astype(int)
    if len(bits) % 2 != 0:
        bits = np.concatenate([bits, np.zeros(1, dtype=int)])
    b = bits.reshape(-1, 2)

    mapping = {
        (0, 0):  1 + 1j,
        (0, 1):  1 - 1j,
        (1, 1): -1 - 1j,
        (1, 0): -1 + 1j,
    }

    symbols = np.array([mapping[tuple(bb)] for bb in b]) / np.sqrt(2)
    return symbols


def qpsk_demodulate(symbols):
    """
    Hard-decision demodulator (nearest quadrant).
    Inverse of qpsk_modulate mapping above.
    """
    symbols = np.array(symbols)
    re = np.real(symbols)
    im = np.imag(symbols)

    bits = []
    for r, i in zip(re, im):
        b0 = 0 if r >= 0 else 1
        b1 = 0 if i >= 0 else 1
        bits.extend([b0, b1])

    return np.array(bits, dtype=int)


# ===============================
# 3. OFDM Mod / Demod
# ===============================

def ofdm_modulate(symbols, n_subc=64, cp_len=16):
    """
    Simple baseband OFDM modulation:
    - Pack symbols to blocks of size n_subc
    - IFFT per block
    - Append CP (cyclic prefix)
    """
    symbols = np.array(symbols, dtype=complex)
    if len(symbols) % n_subc != 0:
        pad = n_subc - (len(symbols) % n_subc)
        symbols = np.concatenate([symbols, np.zeros(pad, dtype=complex)])
    frames = symbols.reshape(-1, n_subc)

    # IFFT
    time_domain = np.fft.ifft(frames, axis=1)
    # Add cyclic prefix
    cp = time_domain[:, -cp_len:]
    with_cp = np.concatenate([cp, time_domain], axis=1)
    return with_cp.reshape(-1)


def ofdm_demodulate(rx_signal, n_subc=64, cp_len=16):
    """
    Inverse of ofdm_modulate:
    - Reshape to blocks
    - Remove CP
    - FFT
    """
    rx_signal = np.array(rx_signal, dtype=complex)
    symb_len = n_subc + cp_len
    assert len(rx_signal) % symb_len == 0, "Length of signal not multiple of OFDM symbol length."
    frames = rx_signal.reshape(-1, symb_len)

    frames_no_cp = frames[:, cp_len:]
    freq_domain = np.fft.fft(frames_no_cp, axis=1)
    return freq_domain.reshape(-1)


# ===============================
# 4. AWGN channel
# ===============================

def add_awgn(signal, snr_db):
    """
    Add complex AWGN (per complex dimension) for given SNR (Es/N0).
    """
    signal = np.array(signal, dtype=complex)
    sig_power = np.mean(np.abs(signal) ** 2)
    snr_linear = 10 ** (snr_db / 10.0)
    noise_power = sig_power / snr_linear
    noise = np.sqrt(noise_power / 2) * (
        np.random.randn(*signal.shape) + 1j * np.random.randn(*signal.shape)
    )
    return signal + noise


# ===============================
# 5. شبیه‌سازی کامل: Hamming + QPSK + OFDM
# ===============================

def simulate_hamming_ofdm(num_bits=4096, snr_db=10, n_subc=64, cp_len=16):
    """
    Full chain:
    bits -> Hamming(7,4) -> QPSK -> OFDM -> AWGN -> OFDM demod -> QPSK demod -> Hamming decode
    Returns BER (bit error rate) on data bits.
    """
    # Generate random bits
    bits = np.random.randint(0, 2, num_bits)

    # Hamming encode
    encoded = hamming74_encode(bits)

    # QPSK modulate
    symbols = qpsk_modulate(encoded)

    # OFDM
    tx_ofdm = ofdm_modulate(symbols, n_subc=n_subc, cp_len=cp_len)

    # Channel: AWGN
    rx_ofdm = add_awgn(tx_ofdm, snr_db)

    # OFDM demod
    rx_symbols = ofdm_demodulate(rx_ofdm, n_subc=n_subc, cp_len=cp_len)
    rx_symbols = rx_symbols[:len(symbols)]  # remove padding

    # QPSK demod
    decoded_code_bits = qpsk_demodulate(rx_symbols)
    decoded_code_bits = decoded_code_bits[:len(encoded)]  # remove padding

    # Hamming decode
    decoded_data_bits = hamming74_decode(decoded_code_bits)
    decoded_data_bits = decoded_data_bits[:num_bits]

    ber = np.mean(bits != decoded_data_bits)
    return ber


# quick sanity check
for snr in [0, 5, 10, 15, 20]:
    ber = simulate_hamming_ofdm(num_bits=4096, snr_db=snr)
    print(f"SNR={snr:2d} dB -> BER ≈ {ber:.5f}")


# ===============================
# 6. نمونه ساده OFDMA دوکاربره
# ===============================

def ofdma_modulate(user_symbols_list, n_subc=64, cp_len=16):
    """
    Very simple OFDMA with 2 users on ONE OFDM symbol:
    - User 0: even subcarriers (0,2,4,...)
    - User 1: odd  subcarriers (1,3,5,...)
    """
    assert len(user_symbols_list) == 2, "This simple demo assumes exactly 2 users."
    u0, u1 = [np.array(u, dtype=complex) for u in user_symbols_list]

    half = n_subc // 2
    if len(u0) < half:
        u0 = np.concatenate([u0, np.zeros(half - len(u0), dtype=complex)])
    if len(u1) < half:
        u1 = np.concatenate([u1, np.zeros(half - len(u1), dtype=complex)])

    freq = np.zeros(n_subc, dtype=complex)
    freq[0::2] = u0[:half]  # even carriers
    freq[1::2] = u1[:half]  # odd carriers

    time_domain = np.fft.ifft(freq)
    cp = time_domain[-cp_len:]
    tx = np.concatenate([cp, time_domain])
    return tx


def ofdma_demodulate(rx_signal, n_subc=64, cp_len=16):
    """
    Inverse of ofdma_modulate (single OFDM symbol).
    Returns [user0_subcarriers, user1_subcarriers]
    """
    rx_signal = np.array(rx_signal, dtype=complex)
    symb_len = n_subc + cp_len
    assert len(rx_signal) == symb_len, "This demo assumes exactly one OFDM symbol."

    frame = rx_signal[cp_len:]
    freq = np.fft.fft(frame)
    u0 = freq[0::2]
    u1 = freq[1::2]
    return [u0, u1]


def simulate_ofdma_two_users(snr_db=15, n_subc=64, cp_len=16):
    """
    Demo: 2 users, each Hamming + QPSK + OFDMA on a single OFDM symbol.
    Returns BER for user0, user1.
    """
    half = n_subc // 2

    # User 0 bits
    bits0 = np.random.randint(0, 2, 4 * half)  # multiple of 4 for Hamming
    enc0 = hamming74_encode(bits0)
    sym0 = qpsk_modulate(enc0)

    # User 1 bits
    bits1 = np.random.randint(0, 2, 4 * half)
    enc1 = hamming74_encode(bits1)
    sym1 = qpsk_modulate(enc1)

    # فقط یک OFDMA symbol، پس به اندازه half subcarrier از هر کاربر استفاده می‌کنیم
    sym0_use = sym0[:half]
    sym1_use = sym1[:half]

    # OFDMA mod
    tx = ofdma_modulate([sym0_use, sym1_use], n_subc=n_subc, cp_len=cp_len)

    # AWGN channel
    rx = add_awgn(tx, snr_db=snr_db)

    # OFDMA demod
    u0_hat, u1_hat = ofdma_demodulate(rx, n_subc=n_subc, cp_len=cp_len)

    # Demod users
    dec0_code_bits = qpsk_demodulate(u0_hat[:len(sym0_use)])
    dec1_code_bits = qpsk_demodulate(u1_hat[:len(sym1_use)])

    # برای سادگی، فقط بخش‌هایی از کد که طولشان مضرب 7 باشد را دیکد می‌کنیم
    L0 = (len(dec0_code_bits) // 7) * 7
    L1 = (len(dec1_code_bits) // 7) * 7

    dec0_data = hamming74_decode(dec0_code_bits[:L0])
    dec1_data = hamming74_decode(dec1_code_bits[:L1])

    # داده‌ی مرجع معادل
    ref0 = hamming74_decode(hamming74_encode(bits0))[:len(dec0_data)]
    ref1 = hamming74_decode(hamming74_encode(bits1))[:len(dec1_data)]

    ber0 = np.mean(ref0 != dec0_data) if len(dec0_data) > 0 else np.nan
    ber1 = np.mean(ref1 != dec1_data) if len(dec1_data) > 0 else np.nan

    return ber0, ber1


ber0, ber1 = simulate_ofdma_two_users(snr_db=15)
print(f"OFDMA demo (2 users, SNR=15 dB): BER_user0 ≈ {ber0:.5f}, BER_user1 ≈ {ber1:.5f}")

ModuleNotFoundError: No module named 'numpy'

In [2]:
python
!pip install numpy

NameError: name 'python' is not defined

In [3]:
!pip install numpy

   ---------------------------------------- 0.0/12.9 MB ? eta -:--:--
   ---------------------------------------- 0.0/12.9 MB ? eta -:--:--
   ---------------------------------------- 0.0/12.9 MB ? eta -:--:--
    --------------------------------------- 0.3/12.9 MB ? eta -:--:--
    --------------------------------------- 0.3/12.9 MB ? eta -:--:--
    --------------------------------------- 0.3/12.9 MB ? eta -:--:--
   - -------------------------------------- 0.5/12.9 MB 407.6 kB/s eta 0:00:31
   - -------------------------------------- 0.5/12.9 MB 407.6 kB/s eta 0:00:31
   -- ------------------------------------- 0.8/12.9 MB 465.9 kB/s eta 0:00:27
   -- ------------------------------------- 0.8/12.9 MB 465.9 kB/s eta 0:00:27
   --- ------------------------------------ 1.0/12.9 MB 504.6 kB/s eta 0:00:24
   --- ------------------------------------ 1.0/12.9 MB 504.6 kB/s eta 0:00:24
   ---- ----------------------------------- 1.3/12.9 MB 516.4 kB/s eta 0:00:23
   ---- -------------------

In [4]:
# %% [markdown]
# شبیه‌سازی Hamming(7,4) + QPSK + OFDM / OFDMA

# %%
import numpy as np

# ===============================
# 1. Hamming(7,4) encoder / decoder
# ===============================

def hamming74_encode(bits):
    """
    Hamming(7,4) systematic encoder
    input:  1D array/list of bits {0,1}
    output: coded bits (len multiple of 7)
    """
    bits = np.array(bits).astype(int)
    if len(bits) % 4 != 0:
        pad = 4 - (len(bits) % 4)
        bits = np.concatenate([bits, np.zeros(pad, dtype=int)])
    data = bits.reshape(-1, 4)

    # Generator matrix G (4x7) for Hamming(7,4) systematic form [I4 | P]
    G = np.array([
        [1, 0, 0, 0, 0, 1, 1],
        [0, 1, 0, 0, 1, 0, 1],
        [0, 0, 1, 0, 1, 1, 0],
        [0, 0, 0, 1, 1, 1, 1]
    ], dtype=int)

    codewords = (data @ G) % 2
    return codewords.reshape(-1)


def hamming74_decode(received):
    """
    Hamming(7,4) hard-decision decoder
    input:  coded bits (len multiple of 7)
    output: estimated data bits (length multiple of 4)
    """
    r = np.array(received).astype(int)
    assert len(r) % 7 == 0, "Length of received must be multiple of 7."
    r = r.reshape(-1, 7)

    # Parity-check matrix H (3x7)
    H = np.array([
        [1, 0, 1, 0, 1, 0, 1],
        [0, 1, 1, 0, 0, 1, 1],
        [0, 0, 0, 1, 1, 1, 1]
    ], dtype=int)

    syndromes = (H @ r.T) % 2  # 3 x n
    syndromes = syndromes.T    # n x 3

    corrected = r.copy()

    # syndrome → error position (1..7), 0 means no error
    syndrome_table = {
        (0, 0, 0): 0,
        (1, 0, 0): 1,
        (0, 1, 0): 2,
        (1, 1, 0): 3,
        (0, 0, 1): 4,
        (1, 0, 1): 5,
        (0, 1, 1): 6,
        (1, 1, 1): 7,
    }

    for i, s in enumerate(syndromes):
        idx = syndrome_table.get(tuple(s.tolist()), 0)
        if idx != 0:
            corrected[i, idx - 1] ^= 1  # flip that bit

    data = corrected[:, 0:4].reshape(-1)
    return data


# ===============================
# 2. QPSK Mod / Demod (Gray)
# ===============================

def qpsk_modulate(bits):
    """
    Gray-coded QPSK
    bits pair: (b0,b1)
    00 ->  1 + 1j
    01 ->  1 - 1j
    11 -> -1 - 1j
    10 -> -1 + 1j
    (normalized by sqrt(2))
    """
    bits = np.array(bits).astype(int)
    if len(bits) % 2 != 0:
        bits = np.concatenate([bits, np.zeros(1, dtype=int)])
    b = bits.reshape(-1, 2)

    mapping = {
        (0, 0):  1 + 1j,
        (0, 1):  1 - 1j,
        (1, 1): -1 - 1j,
        (1, 0): -1 + 1j,
    }

    symbols = np.array([mapping[tuple(bb)] for bb in b]) / np.sqrt(2)
    return symbols


def qpsk_demodulate(symbols):
    """
    Hard-decision demodulator (nearest quadrant).
    Inverse of qpsk_modulate mapping above.
    """
    symbols = np.array(symbols)
    re = np.real(symbols)
    im = np.imag(symbols)

    bits = []
    for r, i in zip(re, im):
        b0 = 0 if r >= 0 else 1
        b1 = 0 if i >= 0 else 1
        bits.extend([b0, b1])

    return np.array(bits, dtype=int)


# ===============================
# 3. OFDM Mod / Demod
# ===============================

def ofdm_modulate(symbols, n_subc=64, cp_len=16):
    """
    Simple baseband OFDM modulation:
    - Pack symbols to blocks of size n_subc
    - IFFT per block
    - Append CP (cyclic prefix)
    """
    symbols = np.array(symbols, dtype=complex)
    if len(symbols) % n_subc != 0:
        pad = n_subc - (len(symbols) % n_subc)
        symbols = np.concatenate([symbols, np.zeros(pad, dtype=complex)])
    frames = symbols.reshape(-1, n_subc)

    # IFFT
    time_domain = np.fft.ifft(frames, axis=1)
    # Add cyclic prefix
    cp = time_domain[:, -cp_len:]
    with_cp = np.concatenate([cp, time_domain], axis=1)
    return with_cp.reshape(-1)


def ofdm_demodulate(rx_signal, n_subc=64, cp_len=16):
    """
    Inverse of ofdm_modulate:
    - Reshape to blocks
    - Remove CP
    - FFT
    """
    rx_signal = np.array(rx_signal, dtype=complex)
    symb_len = n_subc + cp_len
    assert len(rx_signal) % symb_len == 0, "Length of signal not multiple of OFDM symbol length."
    frames = rx_signal.reshape(-1, symb_len)

    frames_no_cp = frames[:, cp_len:]
    freq_domain = np.fft.fft(frames_no_cp, axis=1)
    return freq_domain.reshape(-1)


# ===============================
# 4. AWGN channel
# ===============================

def add_awgn(signal, snr_db):
    """
    Add complex AWGN (per complex dimension) for given SNR (Es/N0).
    """
    signal = np.array(signal, dtype=complex)
    sig_power = np.mean(np.abs(signal) ** 2)
    snr_linear = 10 ** (snr_db / 10.0)
    noise_power = sig_power / snr_linear
    noise = np.sqrt(noise_power / 2) * (
        np.random.randn(*signal.shape) + 1j * np.random.randn(*signal.shape)
    )
    return signal + noise


# ===============================
# 5. شبیه‌سازی کامل: Hamming + QPSK + OFDM
# ===============================

def simulate_hamming_ofdm(num_bits=4096, snr_db=10, n_subc=64, cp_len=16):
    """
    Full chain:
    bits -> Hamming(7,4) -> QPSK -> OFDM -> AWGN -> OFDM demod -> QPSK demod -> Hamming decode
    Returns BER (bit error rate) on data bits.
    """
    # Generate random bits
    bits = np.random.randint(0, 2, num_bits)

    # Hamming encode
    encoded = hamming74_encode(bits)

    # QPSK modulate
    symbols = qpsk_modulate(encoded)

    # OFDM
    tx_ofdm = ofdm_modulate(symbols, n_subc=n_subc, cp_len=cp_len)

    # Channel: AWGN
    rx_ofdm = add_awgn(tx_ofdm, snr_db)

    # OFDM demod
    rx_symbols = ofdm_demodulate(rx_ofdm, n_subc=n_subc, cp_len=cp_len)
    rx_symbols = rx_symbols[:len(symbols)]  # remove padding

    # QPSK demod
    decoded_code_bits = qpsk_demodulate(rx_symbols)
    decoded_code_bits = decoded_code_bits[:len(encoded)]  # remove padding

    # Hamming decode
    decoded_data_bits = hamming74_decode(decoded_code_bits)
    decoded_data_bits = decoded_data_bits[:num_bits]

    ber = np.mean(bits != decoded_data_bits)
    return ber


# quick sanity check
for snr in [0, 5, 10, 15, 20]:
    ber = simulate_hamming_ofdm(num_bits=4096, snr_db=snr)
    print(f"SNR={snr:2d} dB -> BER ≈ {ber:.5f}")


# ===============================
# 6. نمونه ساده OFDMA دوکاربره
# ===============================

def ofdma_modulate(user_symbols_list, n_subc=64, cp_len=16):
    """
    Very simple OFDMA with 2 users on ONE OFDM symbol:
    - User 0: even subcarriers (0,2,4,...)
    - User 1: odd  subcarriers (1,3,5,...)
    """
    assert len(user_symbols_list) == 2, "This simple demo assumes exactly 2 users."
    u0, u1 = [np.array(u, dtype=complex) for u in user_symbols_list]

    half = n_subc // 2
    if len(u0) < half:
        u0 = np.concatenate([u0, np.zeros(half - len(u0), dtype=complex)])
    if len(u1) < half:
        u1 = np.concatenate([u1, np.zeros(half - len(u1), dtype=complex)])

    freq = np.zeros(n_subc, dtype=complex)
    freq[0::2] = u0[:half]  # even carriers
    freq[1::2] = u1[:half]  # odd carriers

    time_domain = np.fft.ifft(freq)
    cp = time_domain[-cp_len:]
    tx = np.concatenate([cp, time_domain])
    return tx


def ofdma_demodulate(rx_signal, n_subc=64, cp_len=16):
    """
    Inverse of ofdma_modulate (single OFDM symbol).
    Returns [user0_subcarriers, user1_subcarriers]
    """
    rx_signal = np.array(rx_signal, dtype=complex)
    symb_len = n_subc + cp_len
    assert len(rx_signal) == symb_len, "This demo assumes exactly one OFDM symbol."

    frame = rx_signal[cp_len:]
    freq = np.fft.fft(frame)
    u0 = freq[0::2]
    u1 = freq[1::2]
    return [u0, u1]


def simulate_ofdma_two_users(snr_db=15, n_subc=64, cp_len=16):
    """
    Demo: 2 users, each Hamming + QPSK + OFDMA on a single OFDM symbol.
    Returns BER for user0, user1.
    """
    half = n_subc // 2

    # User 0 bits
    bits0 = np.random.randint(0, 2, 4 * half)  # multiple of 4 for Hamming
    enc0 = hamming74_encode(bits0)
    sym0 = qpsk_modulate(enc0)

    # User 1 bits
    bits1 = np.random.randint(0, 2, 4 * half)
    enc1 = hamming74_encode(bits1)
    sym1 = qpsk_modulate(enc1)

    # فقط یک OFDMA symbol، پس به اندازه half subcarrier از هر کاربر استفاده می‌کنیم
    sym0_use = sym0[:half]
    sym1_use = sym1[:half]

    # OFDMA mod
    tx = ofdma_modulate([sym0_use, sym1_use], n_subc=n_subc, cp_len=cp_len)

    # AWGN channel
    rx = add_awgn(tx, snr_db=snr_db)

    # OFDMA demod
    u0_hat, u1_hat = ofdma_demodulate(rx, n_subc=n_subc, cp_len=cp_len)

    # Demod users
    dec0_code_bits = qpsk_demodulate(u0_hat[:len(sym0_use)])
    dec1_code_bits = qpsk_demodulate(u1_hat[:len(sym1_use)])

    # برای سادگی، فقط بخش‌هایی از کد که طولشان مضرب 7 باشد را دیکد می‌کنیم
    L0 = (len(dec0_code_bits) // 7) * 7
    L1 = (len(dec1_code_bits) // 7) * 7

    dec0_data = hamming74_decode(dec0_code_bits[:L0])
    dec1_data = hamming74_decode(dec1_code_bits[:L1])

    # داده‌ی مرجع معادل
    ref0 = hamming74_decode(hamming74_encode(bits0))[:len(dec0_data)]
    ref1 = hamming74_decode(hamming74_encode(bits1))[:len(dec1_data)]

    ber0 = np.mean(ref0 != dec0_data) if len(dec0_data) > 0 else np.nan
    ber1 = np.mean(ref1 != dec1_data) if len(dec1_data) > 0 else np.nan

    return ber0, ber1


ber0, ber1 = simulate_ofdma_two_users(snr_db=15)
print(f"OFDMA demo (2 users, SNR=15 dB): BER_user0 ≈ {ber0:.5f}, BER_user1 ≈ {ber1:.5f}")

SNR= 0 dB -> BER ≈ 0.14624
SNR= 5 dB -> BER ≈ 0.01465
SNR=10 dB -> BER ≈ 0.00000
SNR=15 dB -> BER ≈ 0.00000
SNR=20 dB -> BER ≈ 0.00000
OFDMA demo (2 users, SNR=15 dB): BER_user0 ≈ 0.00000, BER_user1 ≈ 0.00000
